In [5]:
import sys
sys.path.append('../scripts')

import pandas as pd
import numpy as np

import matplotlib.pyplot as plt

from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import roc_auc_score,accuracy_score, f1_score
from sklearn.model_selection import KFold
from sklearn.model_selection import cross_val_score
from matplotlib.colors import ListedColormap
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import Perceptron

from sklearn.metrics import ConfusionMatrixDisplay

import features as ft
import train as tr
import test as ts

In [6]:
### Import data sources ###

# Training data
train = pd.read_csv('../data/train.csv')

#Previously purchased subscriptions by account
subscriptions = pd.read_csv('../data/subscriptions.csv')
#display(subscriptions.head())

# Location info for each patron and donation history
accounts = pd.read_csv('../data/account2.csv')
#display(accounts.head())

# Previous concerts by season
concerts = pd.read_csv('../data/concerts.csv')
#display(concerts.head())

# List of planned concert sets for the 2014-15 season
planned_concerts = pd.read_csv('../data/concerts_2014-15.csv')
#display(planned_concerts.head())

# Previously purchased tickets by account
tickets = pd.read_csv('../data/tickets_all.csv')
#display(tickets.head())

# Location and demographic information for zipcodes
zipcodes = pd.read_csv('../data/zipcodes.csv')
#display(zipcodes.head())

In [7]:
def train_and_test():
    df = ft.get_features(train, accounts, subscriptions)
    df = ft.clean_account_data(df)

    display(df.head())

    X = df.drop(columns=['account.id_x', 'account.id_y', 'label', 'ID'])
    y = df['label']

    X_train,X_test,y_train,y_test = train_test_split(X, y, random_state=0,test_size=0.2)

    best_model = tr.get_best_model(X_train, y_train)
    ts.get_test_score(X_test, y_test, best_model)

In [8]:
train_and_test()

There are 6841 rows that have NaN zip code and city name


,account.id_x,label,ID,account.id_y,billing.zip.code,billing.city,amount.donated.2013,amount.donated.lifetime,no.donations.lifetime,num_subscriptions
0,001i000000NuRo3,0,001i000000NuRo3,001i000000NuRo3,NaN,NaN,0.0,0.0,0,0.0
1,001i000000NuRxd,0,001i000000NuRxd,001i000000NuRxd,NaN,NaN,0.0,0.0,0,0.0
2,001i000000NuQGN,0,001i000000NuQGN,001i000000NuQGN,94597,Walnut Creek,0.0,0.0,0,0.0
3,001i000000NuPfL,0,001i000000NuPfL,001i000000NuPfL,94005,Brisbane,0.0,0.0,0,1.0
4,001i000000NuQkP,0,001i000000NuQkP,001i000000NuQkP,94610,Oakland,0.0,0.0,0,0.0


Found best KNN with optimal neighbors: 10
KNN  model AUROC:  0.6927154987063712
Linear SVC  model AUROC:  0.7154209323618111
RBF SVC  model AUROC:  0.6835658706933954
Perceptron  model AUROC:  0.6641566977957452
Best performance model is  Linear SVC


AttributeError: module 'test' has no attribute 'get_test_score'